# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver correspondente ao navegador que está sendo utilizado e colar o arquivo .exe na pasta do anaconda

- Diferenças entre pyautogui e selenium

- pyautogui -> automatizar mouse, teclado e tela
    - são automações que impedem que você utilize o computador
    - funciona em qualquer ferramenta, sistema ou programa
    - caso a tela mude, o pyautogui para de funcionar
- selenium -> automatizar a web/internet
    - só funciona para a internet
    - funciona mesmo com outra ação sendo executada no computador
    - funciona mesmo que a posição dos elementos da tela mude

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# para rodar o chrome em 2º plano
# from selenium.webdriver.chrome.options import Options
# chrome_options = Options()
# chrome_options.headless = True # also works
# nav = webdriver.Chrome(options=chrome_options)

nav = webdriver.Chrome() # abre um novo navegador

In [18]:
# pesquisar cotação dolar
nav.get("https://www.google.com/") # digita endereço na barra
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar") # escreve um texto no campo de busca do google - acesso ao campo por meio do 'Inspecionar', em seguida 'Copy XPath' 
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # tecla enter
cotacao_dolar = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')  # obter cotação do dólar
print(cotacao_dolar)

5.438566


In [19]:
# pesquisar cotação dolar
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')  # obter cotação do euro
print(cotacao_euro)

6.537129139


In [20]:
# pesquisar cotação do ouro
nav.get("https://www.melhorcambio.com/") # digita endereço na barra
aba_original = nav.window_handles[0]
nav.find_element_by_xpath('//*[@id="commodity-hoje"]/tbody/tr[2]/td[2]/a/img').click() # clica no ícone da cotação do ouro
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova) 
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value') # obter cotação do ouro
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

309.14


### Agora vamos atualizar a nossa base de preços com as novas cotações

- Importando a base de dados

In [21]:
import pandas as pd

tabela_produtos = pd.read_excel("Produtos.xlsx")
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [22]:
# atualiza os valores das cotações na tabela
tabela_produtos.loc[tabela_produtos['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)

tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original'] # atualiza a coluna do preço base real com base nas novas cotações
tabela_produtos['Preço Final'] = tabela_produtos['Ajuste'] * tabela_produtos['Preço Base Reais'] # atualiza o preço final
# tabela_produtos["Preço Final"] = tabela_produtos["Preço Final"].map("{:.2f}".format) -> para melhorar a formatação dos valores
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.438566,5438.511614,1.40,7613.916260
1,Carro Renault,4500.00,Euro,6.537129,29417.081126,2.00,58834.162251
2,Notebook Dell,899.99,Dólar,5.438566,4894.655014,1.70,8320.913524
3,IPhone,799.00,Dólar,5.438566,4345.414234,1.70,7387.204198
4,Carro Fiat,3000.00,Euro,6.537129,19611.387417,1.90,37261.636092
5,Celular Xiaomi,480.48,Dólar,5.438566,2613.122192,2.00,5226.244383
6,Joia 20g,20.00,Ouro,309.140000,6182.800000,1.15,7110.220000


### Agora vamos exportar a nova base de preços atualizada

In [23]:
tabela_produtos.to_excel("Produtos Atualizados.xlsx", index = False) # index = false -> excluir os índices novos (opcional)